In [1]:
from gurobipy import Model

# Test Gurobi installation
model = Model("test")
model.optimize()


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [2]:
import gurobipy as gp

options = {
    "WLSACCESSID": "dc1ebca6-cb85-4ac2-a146-8d704770ea00",
    "WLSSECRET": "224cf33d-caa0-4b3e-9a5c-7bd4dbeb9da1",
    "LICENSEID": 2585862,
}
with gp.Env(params=options) as env, gp.Model(env=env) as model:
    # Formulate problem
    model.optimize()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2585862
Academic license 2585862 - for non-commercial use only - registered to ch___@northeastern.edu
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2585862 - for non-commercial use only - registered to ch___@northeastern.edu
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [3]:
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

In [5]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load passenger data
passenger_data = pd.read_csv('testing.csv')

# Extract coordinates and IDs
n = len(passenger_data)
xc = passenger_data['Latitude'].values
yc = passenger_data['Longitude'].values
ids = passenger_data['PassengerID'].values
languages = passenger_data['language'].values  

vehicle_languages = [1] * 7 + [0]  # Vehicle language abilities: 1=Chinese, 0=English

# Mapping vehicle indices to languages
num_vehicles = len(vehicle_languages)
vehicle_indices = range(1, num_vehicles + 1)  # Vehicle indices
vehicle_languages_map = {vehicle_indices[i]: vehicle_languages[i] for i in range(num_vehicles)}


# Define depot coordinates (you can adjust this to a central location)
depot_latitude = 42.3527  # Example: Linden Street
depot_longitude = -71.1280
xc = np.append([depot_latitude], xc)
yc = np.append([depot_longitude], yc)


In [6]:
print(passenger_data.columns)

## passenger_data['language'] = np.random.randint(0, 2, size=len(passenger_data))


Index(['PassengerID', 'ResidentialPlace', 'Latitude', 'Longitude', 'language'], dtype='object')


In [7]:
# Define vehicle capacities
vehicle_capacities = [14] * 7 + [4]
num_vehicles = len(vehicle_capacities)
Q = max(vehicle_capacities)  # Maximum capacity (used in constraints)



In [8]:
# Vehicle and client sets
N = [i for i in range(1, n+1)]  # Client IDs
V = [0] + N                    # Include the depot
A = [(i, j) for i in V for j in V if i != j]  # All arcs


In [9]:
# Costs (distances between nodes)
c = {(i, j): np.hypot(xc[i] - xc[j], yc[i] - yc[j]) for i, j in A}

# Demand for each passenger
q = {i: 1 for i in N}

In [10]:
# Optimization Model
mdl = Model('CVRP_Language')
x = mdl.addVars(A, vtype=GRB.BINARY)  # Decision variable: arc usage
u = mdl.addVars(N, vtype=GRB.CONTINUOUS)  # Load of each client
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(quicksum(x[a] * c[a] for a in A))

In [11]:
# Constraints
mdl.addConstrs(quicksum(x[i, j] for j in V if j != i) == 1 for i in N)  # Each client is visited once
mdl.addConstrs(quicksum(x[j, i] for j in V if j != i) == 1 for i in N)  # Each client is departed from once
mdl.addConstrs((x[i, j] == 1) >> (u[i] + q[i] == u[j]) for i, j in A if i != 0 and j != 0)  # Load flow
mdl.addConstrs(u[i] >= q[i] for i in N)  # Demand met
mdl.addConstrs(u[i] <= Q for i in N)  # Capacity respected
mdl.addConstr(quicksum(q[i] for i in N) <= sum(vehicle_capacities))  # Total demand fits capacity

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Language constraints
for i in N:
    passenger_lang = languages[i - 1]  # Passenger language preference (1=Chinese, 0=English)
    mdl.addConstr(
        quicksum(
            x[i, j] for j in V if j != i and j in vehicle_indices and vehicle_languages_map[j] == passenger_lang
        ) == 1,
        name=f"Lang_Constraint_Passenger_{i}"
    )


In [13]:
# Ad Hoc Request Function
def handle_ad_hoc_request(ad_hoc_requests, passenger_data):
    """
    Updates the passenger data for ad hoc requests by replacing their residential locations
    with their requested destinations.
    :param ad_hoc_requests: List of tuples [(PassengerID, latitude, longitude), ...]
    :param passenger_data: Existing DataFrame of passengers.
    :return: Updated passenger data.
    """
    global xc, yc, ids, n
    
    for request in ad_hoc_requests:
        passenger_id, latitude, longitude = request
        
        # Check if the PassengerID exists in the data
        if passenger_id in passenger_data['PassengerID'].values:
            # Update the passenger's destination
            passenger_data.loc[passenger_data['PassengerID'] == passenger_id, ['Latitude', 'Longitude']] = latitude, longitude
        else:
            print(f"PassengerID {passenger_id} not found!")

In [14]:
# Update coordinates and IDs
xc = np.append([depot_latitude], passenger_data['Latitude'].values)
yc = np.append([depot_longitude], passenger_data['Longitude'].values)
ids = passenger_data['PassengerID'].values
n = len(passenger_data) - 1  # Exclude depot




In [15]:
from folium.plugins import MarkerCluster
from folium import DivIcon



def visualize_routes(active_arcs, xc, yc, ids, depot_latitude, depot_longitude, passenger_data):
    """
    Visualize the optimized routes using Folium.
    :param active_arcs: List of active arcs (routes) from the optimization.
    :param xc: Array of x-coordinates (latitudes).
    :param yc: Array of y-coordinates (longitudes).
    :param ids: List of passenger IDs.
    :param depot_latitude: Latitude of the depot.
    :param depot_longitude: Longitude of the depot.
    """
    # Create a map centered around the depot
    route_map = folium.Map(location=[depot_latitude, depot_longitude], zoom_start=13)

    # Add the depot marker
    folium.Marker(
        location=[depot_latitude, depot_longitude],
        popup="Depot",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(route_map)


    # Add a cluster for passenger markers
    marker_cluster = MarkerCluster().add_to(route_map)

    # Group passengers by location
    location_groups = {}
    for i in range(1, len(ids)):  # Skip the depot (index 0)
        loc = (xc[i], yc[i])
        if loc not in location_groups:
            location_groups[loc] = []
        location_groups[loc].append(ids[i])

    # Add markers for each unique location with passenger count on the icon
    for loc, passengers in location_groups.items():
        passenger_count = len(passengers)
        popup_content = f"Passengers: {', '.join(map(str, passengers))}"

        # Custom icon to show the passenger count
        folium.Marker(
            location=loc,
            popup=popup_content,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="font-size: 12px; color: white; background: blue; '
                     f'border-radius: 50%; text-align: center; width: 24px; height: 24px; line-height: 24px;">'
                     f'{passenger_count}</div>'
            )
        ).add_to(route_map)

    # Draw routes
    for i, j in active_arcs:
        # Determine the vehicle's language (assume `i` represents the vehicle index)
        driver_language = vehicle_languages[i]
        route_color = "blue" if driver_language == 0 else "green"  # Blue for English-speaking driver, green otherwise
        
        folium.PolyLine(
            locations=[[xc[i], yc[i]], [xc[j], yc[j]]],
            color=route_color,
            weight=2.5
        ).add_to(marker_cluster)

    # Display the map
    route_map.save('optimized_routes.html')
    print("Map saved as 'optimized_routes.html'. Open this file in your browser to view the routes.")
   


In [16]:
passenger_data = pd.read_csv('testing.csv')

# CLI Interface
def main():
    global passenger_data, mdl
    print("Welcome to the Shuttle Scheduling System")
    
    while True:
        print("\n1. Optimize Schedule")
        print("2. Add Ad Hoc Request")
        print("3. Exit")
        choice = int(input("Choose an option: "))

        if choice == 1:
            print("Starting optimization...")
            mdl.Params.TimeLimit = 20  # Limit optimization to 10 seconds
            mdl.Params.MIPGap = 0.3   # Allow solutions within 20% of optimal
            mdl.Params.OutputFlag = 1  # Enable solver logs
            mdl.optimize()
            print("Optimization complete.")

            if mdl.status == 2 or mdl.status == 3:
                try:
                    # Extract active arcs
                    active_arcs = [a for a in A if x[a].x > 0.99]  # Check if the variable is part of the solution
                    print(f"Total optimized distance: {mdl.ObjVal}")

                    # Visualize the routes
                    visualize_routes(active_arcs, xc, yc, ids, depot_latitude, depot_longitude, passenger_data)
                except AttributeError:
                    print("Error: Variables have no solution values. Check the model constraints.")
            else:
                print("No optimal or feasible solution found!")

        elif choice == 2:
            passenger_id = input("Enter PassengerID: ")
            latitude = float(input("Enter new destination latitude: "))
            longitude = float(input("Enter new destination longitude: "))
            passenger_data = handle_ad_hoc_request([(passenger_id, latitude, longitude)], passenger_data)
            print(f"Ad hoc request for {passenger_id} processed successfully!")

            # Update variables and reinitialize the model
            xc = np.append([depot_latitude], passenger_data['Latitude'].values)
            yc = np.append([depot_longitude], passenger_data['Longitude'].values)
            ids = passenger_data['PassengerID'].values
            languages = passenger_data['language'].values  # Update the language column
            n = len(passenger_data) - 1  # Exclude depot
            
            mdl = reinitialize_model()
            
        elif choice == 3:
            print("Exiting the system. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


main()



Welcome to the Shuttle Scheduling System

1. Optimize Schedule
2. Add Ad Hoc Request
3. Exit
Starting optimization...
Set parameter TimeLimit to value 20
Set parameter MIPGap to value 0.3
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  20
MIPGap  0.3

Optimize a model with 196 rows, 1599 columns and 3356 nonzeros
Model fingerprint: 0xb0948a37
Model has 1482 simple general constraints
  1482 INDICATOR
Variable types: 39 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 84 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations)